#  Building a database for crime reports

In this project, we will build a database for storing data related to crimes that occurred in Boston. We will use Postgres, and in the process we will
- create a database and manage database roles
- create database schemas and tables with the proper datatypes
- load data from CSV files into database tables.

We use the dataset `boston.csv` which has seven columns:
1. `incident_number` (ID of the crime)
2. `offense_code` (numeric identifier code, offense)
3. `description` (description of the incident)
4. `date` (date in YY-MM-DD format)
5. `day_of_the_week` (weekday)
6. `lat` (location, latitude)
7. (`lon`) (location, longitude)

The goal of this guided project is to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `boston.csv` file. We will be creating the table inside a schema named `crimes`. We will also create the `readonly` and `readwrite` groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups. 

## Creating the Crime Database

We will start by creating a database for storing our crime data as well as a schema for containing the tables. 

In [1]:
import psycopg2
import csv

conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

ProgrammingError: database "crime_db" already exists


In the next step, we will add a schema called `crimes`.

In [2]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA IF NOT EXISTS crimes;")
conn.commit()
conn.close()

## Obtaining the Column Names and Sample

We now have a database and a schema — we're ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can easily select the right datatypes to use in our table. Let's create the header row and sperate it from the rest of the data.

In [3]:
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

We now have access to the column headers and the first row of data. This will help us when we create the table. We won't have to remember the names of the columns, and using the first row of data, we can easily recall what kind of data is in each column.

Before we create a table to store the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — `get_col_set()` — that, given the name of a CSV file and a column index (starting at 0), computes a Python set with all distinct values contained in that column. 

In [4]:
def get_col_set(csv_filename, col_index):
    """Takes in a CSV file and a column index, fetches the distinct values in a column
    and returns them as a set."""
    with open(csv_filename) as file:  # open the file
        reader = csv.reader(file)  # create a reader object
        next(reader)  # skip the header row
        return {row[col_index] for row in reader}  # return a set of the distinct values in the column        

In [5]:
# Test the function
get_col_set("boston.csv", 2)

{'A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.',
 'A&B ON POLICE OFFICER',
 'ABDUCTION - INTICING',
 'AFFRAY',
 'AIRCRAFT INCIDENTS',
 'ANIMAL ABUSE',
 'ANIMAL CONTROL - DOG BITES - ETC.',
 'ANIMAL INCIDENTS',
 'ANNOYING AND ACCOSTIN',
 'ANNOYING AND ACCOSTING',
 'ARSON',
 'ASSAULT & BATTERY',
 'ASSAULT & BATTERY D/W - KNIFE',
 'ASSAULT & BATTERY D/W - OTHER',
 'ASSAULT & BATTERY D/W - OTHER ON POLICE OFFICER',
 'ASSAULT - AGGRAVATED',
 'ASSAULT - AGGRAVATED - BATTERY',
 'ASSAULT - SIMPLE',
 'ASSAULT D/W - KNIFE ON POLICE OFFICER',
 'ASSAULT D/W - OTHER',
 'ASSAULT SIMPLE - BATTERY',
 'AUTO THEFT',
 'AUTO THEFT - LEASED/RENTED VEHICLE',
 'AUTO THEFT - MOTORCYCLE / SCOOTER',
 'AUTO THEFT - OUTSIDE - RECOVERED IN BOSTON',
 'AUTO THEFT - RECOVERED IN BY POLICE',
 'AUTO THEFT LEASE/RENT VEHICLE',
 'AUTO THEFT OTHER',
 'B&E NON-RESIDENCE DAY - FORCIBLE',
 'B&E NON-RESIDENCE DAY - NO FORCE',
 'B&E NON-RESIDENCE DAY - NO PROP TAKEN',
 'B&E NON-RESIDENCE NIGHT - FORCE',
 'B&E RESIDENCE DAY - N

It seems to work. Now we will compute for each column how many distinct values it contains by measuring the length of each set.

In [6]:
set_lengths = []
for index in range(7):
    length = len(get_col_set("boston.csv", index))
    set_lengths.append(length)
    
print(set_lengths)

[298329, 219, 239, 1177, 7, 18177, 18177]


We find that there are 298,329 incidents, 219 offense codes, 239 descriptions, 1177 dates, 7 weekdays, 18177 latitudes, and 18177 longitudes.

## Finding the Maximum Length

With the auxilliary function, we can compute the number of distinct values for each column. Columns with a low number of distinct values tend to be good candidates for enumerated datatypes.

Another important aspect is to know the longest word in any column containing textual data. We can actually use the previous function for computing this. Let's compute the maximum length of each value in the description column.

In [7]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [8]:
col_values = get_col_set("boston.csv", 2)  # run the function on the second column, assign to col_values
item_lengths = [len(_) for _ in col_values]  # go through col_values, determine the length of each item and assign it to the list
max_length = max(item_lengths)  # calculate the maximum length of an item in col_values
print(item_lengths[:4])  # print first 5 item lengths
print(max_length)  # print the maximum item length

[42, 40, 11, 33]
58


Ok, so the longest item in the `description` column can have a length of 58 characters.

## Creating the Table

We will now create a table named `boston_crimes` inside the `crimes` schema of the `crime_db` database. We will use the information about the unique items and maximum lengths to select the appropriate datatypes.

In [ ]:
print(col_headers)
print(first_row)

Given the data that we have and each of the lengths, the following datatypes make sense:

- `incident_number` - integer
- `offense_code` - integer; too many distinct codes to be useful for the enumerated type
- `description`- varchar(60); the maximum item length is 58, and 60 will give a ssmall buffer
- `date` date (YYYY-MM-DD)
- `day_of_the_week` - enumerated type; only 7 distinct days makes it suited for enumerated
- `lat` - decimal(10,8); 10 digits in total, 8 after the decimal point
- `long` - decimal(10,8); 10 digits in total, 8 after the decimal point

In [9]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()

# Drop the existing weekday_enum type if it exists
cur.execute("DROP TYPE IF EXISTS weekday_enum CASCADE;")

# create enumerated datatype
cur.execute("CREATE TYPE weekday_enum AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

# Create the table
cur.execute("""
CREATE TABLE IF NOT EXISTS crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(60),
    date DATE,
    day_of_the_week weekday_enum,
    lat DECIMAL(10, 8),
    long DECIMAL(10, 8));
""")

conn.commit()
conn.close()

## Loading the Data

Now that we have created the table, we can load the data into it. We will use the `cursor.copy_expert()` method. First, though, we have to remove duplicate rows from the csv file.

In [10]:
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()

# Remove duplicates from the CSV file
unique_rows = {}
with open("boston.csv", "r") as infile, open("boston_unique.csv", "w", newline='') as outfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
    writer.writeheader()
    for row in reader:
        if row['incident_number'] not in unique_rows:
            unique_rows[row['incident_number']] = row
            writer.writerow(row)

with open("boston_unique.csv", "r") as f:
    try:
        cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
    except psycopg2.Error as e:
        print("Error: {}".format(e))
        conn.rollback()
    else:
        conn.commit()

cur.execute("SELECT COUNT(*) FROM crimes.boston_crimes")
row_count = cur.fetchone()[0]
print(row_count)

conn.close()

Error: extra data after last expected column
CONTEXT:  COPY boston_crimes, line 2: "1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.35779134,-71.13937053"

0


## Revoking Public Privileges

Our database is starting to look good! We have created a database with a schema inside it to hold data about crimes. We selected the right datatypes for storing the data, created a table, and loaded the CSV containing crimes about Boston.

Now it's time to handle users. Our goal is to create the two user groups that we have learned about: `readonly` and `readwrite`. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

We thus revoke the privileges of the `public`group on the `public`schema.
Then we revoke all privileges of `public` on the `crime_db` database.

In [11]:
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
conn.commit()
conn.close()

## Creating User Groups

We've made sure that we aren't going to inadvertently inherit privileges from the `public` group. The next step is to create our two user groups. 

In [13]:
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()
cur.execute("CREATE GROUP readonly NOLOGIN ;")
cur.execute("CREATE GROUP readwrite NOLOGIN ;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()
conn.close()

## Creating Users

Now we just need to create the users!

In [14]:
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()

cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

conn.commit()
conn.close()

## Testing

Let's test if everything is configured as expected.

In [17]:
# Close the old connection to test with a brand new connection.
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# Check users and groups.
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
